In [98]:
import requests
from bs4 import BeautifulSoup
import time

resp = requests.get('https://www.gastronom.ru/recipe/')
soup = BeautifulSoup(resp.text, 'html.parser')

recipes_urls = []
for n in range(1,19):
    print(n)
    resp = requests.get('https://www.utkonos.ru/recipes/category?page={}'.format(n))
    soup = BeautifulSoup(resp.text, 'html.parser')
    rec = soup("a", class_= "title")
    for d in rec:
        recipes_urls.append('https://www.utkonos.ru'+d['href'])
    time.sleep(2)
    
#https://www.utkonos.ru/recipes/category?page=2

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [1]:
#сслыки на разделы с рецептами
import requests
from bs4 import BeautifulSoup
import time

resp = requests.get('https://www.gastronom.ru/recipe/')
soup = BeautifulSoup(resp.text, 'html.parser')

cat_urls = []
recs = soup.select("div.popular > a[href^='/recipe']")

for d in recs:
    cat_urls.append('https://www.gastronom.ru'+d['href'])
cat_urls

['https://www.gastronom.ru/recipe/group/2067/zakuski',
 'https://www.gastronom.ru/recipe/group/3225/recepty-salatov',
 'https://www.gastronom.ru/recipe/group/1129/recepty-supov',
 'https://www.gastronom.ru/recipe/group/3211/vtorye-blyuda',
 'https://www.gastronom.ru/recipe/group/1142/vypechka',
 'https://www.gastronom.ru/recipe/group/1645/kashi',
 'https://www.gastronom.ru/recipe/group/1105/zapekanki',
 'https://www.gastronom.ru/recipe/group/1060/pirogi',
 'https://www.gastronom.ru/recipe/group/3224/recepty-tortov',
 'https://www.gastronom.ru/recipe/group/1122/deserty',
 'https://www.gastronom.ru/recipe/group/1132/recepty-napitkov-domashnie-napitki']

In [4]:
def lastpage(url):
    url = url + '?&page=999'
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    if soup.select("div.pagination > a:nth-of-type(7)"):
        return soup.select("div.pagination > a:nth-of-type(7)")[0].text
    else:
        return '1'

for cat in cat_urls:
    print(cat.split('/')[6] + ' ' + lastpage(cat) + ' страниц')

zakuski 144 страниц
recepty-salatov 98 страниц
recepty-supov 59 страниц
vtorye-blyuda 1 страниц
vypechka 1 страниц
kashi 8 страниц
zapekanki 19 страниц
pirogi 1 страниц
recepty-tortov 29 страниц
deserty 1 страниц
recepty-napitkov-domashnie-napitki 1 страниц


In [20]:
#забыл про переход по страницам
import re

def lastpage(url):
    url = url + '?&page=999'
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    if soup.select("div.pagination > a:nth-of-type(7)"):
        return soup.select("div.pagination > a:nth-of-type(7)")[0].text
    else:
        return '1'
    
with open('recipes_gastronom.csv', 'w') as f:
    f.write('name|ingredients|text\n')
    
k = 1 #категория 
url = ''
rec_urls = []
count = 0
for cat_url in cat_urls:
    for x in range(1,int(lastpage(cat_url))):
        resp = requests.get(cat_url + '?&page={}'.format(x)) # x - номер страницы в категории
        soup = BeautifulSoup(resp.text, 'html.parser')
        rec = soup.select("div.row.feed h3 > a")
        n = 1 #номер рецепта на странице
        
        for el in rec:
            rec_text = ''
            url = 'https://www.gastronom.ru'+el['href']
            rec_urls.append(url)
            if  rec_urls.count(url) > 1:
                print (url, rec_urls.count(url))
                continue
            resp = requests.get(url)
            print (k, x, n, url)
            soup = BeautifulSoup(resp.text, 'html.parser')
            title = soup.find("h1").text
            steps = soup.select("div.step.instruction p")
            for step in steps:
                rec_text += step.text.replace('\r\n', '').replace('\xa0', ' ')
            rec_text = re.sub(r"\s+", u" ", rec_text).strip()
            ingredients_collection = soup.select("li.ingredient")
            ingredients = [i.text.replace('\xa0', ' ') for i in ingredients_collection]
            with open('recipes_gastronom.csv', 'a') as f:
                f.write('{}|{}|{}\n'.format(title, ','.join(ingredients), rec_text))
            n += 1
            count += 1
            time.sleep(2)
    k += 1
print ('Всего ссылок: ', len(rec_urls), ' Уникальных: ', count)

1 1 1 https://www.gastronom.ru/recipe/15558/prazdnichnyj-rulet-iz-lososya
1 1 2 https://www.gastronom.ru/recipe/35650/rulety-iz-lavasha-s-salatom-iz-fasoli-i-avokado
1 1 3 https://www.gastronom.ru/recipe/31670/krem-iz-avokado
1 1 4 https://www.gastronom.ru/recipe/11541/rulety-iz-vetchiny-s-syrnoj-nachinkoj
1 1 5 https://www.gastronom.ru/recipe/24858/avokado-vo-fritjure
1 1 6 https://www.gastronom.ru/recipe/21098/goryachaya-buzhenina
1 1 7 https://www.gastronom.ru/recipe/20451/kurinyj-zhyulen-s-gribami
1 1 8 https://www.gastronom.ru/recipe/13314/rulety-iz-lavasha
1 1 9 https://www.gastronom.ru/recipe/14894/zhyulen-iz-gribov
1 1 10 https://www.gastronom.ru/recipe/13279/forshmak
1 1 11 https://www.gastronom.ru/recipe/11551/feta-s-chesnokom-i-zelenyu
1 1 12 https://www.gastronom.ru/recipe/10595/ogurcy-solenye
1 1 13 https://www.gastronom.ru/recipe/10133/myasnoj-rulet-s-perepelinymi-yajcami
1 1 14 https://www.gastronom.ru/recipe/1920/kurinyj-zhyulen
1 1 15 https://www.gastronom.ru/recipe/30

1 4 2 https://www.gastronom.ru/recipe/39816/salat-s-yaichnymi-serdechkami
1 4 3 https://www.gastronom.ru/recipe/41991/zapechennyj-krem-iz-pecheni-s-gribami-pod-brusnichnym-zhele
1 4 4 https://www.gastronom.ru/recipe/41989/baklazhany-puchholiya
1 4 5 https://www.gastronom.ru/recipe/41972/rulet-iz-farsha-s-yajcom-i-lukom
1 4 6 https://www.gastronom.ru/recipe/41985/krab-s-tomatnym-terrinom-i-tryufelnym-maslom
1 4 7 https://www.gastronom.ru/recipe/41984/brusketta-s-avokado-i-pomidorami
1 4 8 https://www.gastronom.ru/recipe/41983/salat-iz-paltusa-s-bakinskimi-pomidorami
1 4 9 https://www.gastronom.ru/recipe/41974/file-iz-dikogo-sibasa-krudo-
1 4 10 https://www.gastronom.ru/recipe/41973/yajca-po-shotlandski
1 4 11 https://www.gastronom.ru/recipe/3082/kanape
1 4 12 https://www.gastronom.ru/recipe/41964/rulet-s-myasnym-farshem-i-mindalem
1 4 13 https://www.gastronom.ru/recipe/41963/pide
1 4 14 https://www.gastronom.ru/recipe/41951/hot-dog-s-chili
1 4 15 https://www.gastronom.ru/recipe/13276/ba

1 6 36 https://www.gastronom.ru/recipe/41095/kartofelnye-bashenki
1 7 1 https://www.gastronom.ru/recipe/41096/tartar-iz-chorizo-s-syrom-i-tomatami
1 7 2 https://www.gastronom.ru/recipe/41080/zapechennyj-syr-kamamber-s-chesnokom-i-timyanom
1 7 3 https://www.gastronom.ru/recipe/41064/veganskie-vegetarianskie-chapati-s-syrom-i-ovoshchami-na-kostre
1 7 4 https://www.gastronom.ru/recipe/41034/zelenyj-salat-s-utkoj-klubnikoj-i-avokado
1 7 5 https://www.gastronom.ru/recipe/41002/bystrye-goryachie-buterbrody
1 7 6 https://www.gastronom.ru/recipe/40773/domashnie-svinye-kolbaski-s-adzhikoj
1 7 7 https://www.gastronom.ru/recipe/40772/tapas-primus-s-malosolenym-lososem--s-kopchenym-lososem-s-govyazhej-vyrezkoj-i-pechenym-percem
1 7 8 https://www.gastronom.ru/recipe/40955/rulet-iz-syra-vetchiny-i-shampinonov
1 7 9 https://www.gastronom.ru/recipe/40948/karpachcho-iz-osminoga-s-poke-iz-avokado-i-tomatov
1 7 10 https://www.gastronom.ru/recipe/40937/tort-zakusochnyj-na-vafelnyh-korzhah-k-9-maya
1 7 11 

1 9 35 https://www.gastronom.ru/recipe/40230/buterbrod-so-stejkom-iz-tykvy-rzhanym-hlebom-i-pikantnym-sousom
1 9 36 https://www.gastronom.ru/recipe/40227/kartofelnye-korzinochki-s-krasnoj-chechevicej
1 10 1 https://www.gastronom.ru/recipe/40233/kartofel-zapechennyj-po-turecki
1 10 2 https://www.gastronom.ru/recipe/40199/peperonata
1 10 3 https://www.gastronom.ru/recipe/40198/shampinony-v-hrustyashchem-klyare
1 10 4 https://www.gastronom.ru/recipe/40196/marinovannye-kabachki
1 10 5 https://www.gastronom.ru/recipe/40189/zelenyj-rulet-s-tvorozhno-syrnym-kremom-i-klubnikoj
1 10 6 https://www.gastronom.ru/recipe/40186/adzhapsandali
1 10 7 https://www.gastronom.ru/recipe/40178/ruletiki-iz-lavasha-s-kartofelem-gribami-i-zelenyu
1 10 8 https://www.gastronom.ru/recipe/40177/marinovannaya-kapusta-s-kurkumoj
1 10 9 https://www.gastronom.ru/recipe/40176/pashtet-iz-fasoli-s-kinzoj-i-paprikoj
1 10 10 https://www.gastronom.ru/recipe/40175/fasol-v-chesnochnom-souse
1 10 11 https://www.gastronom.ru/rec

1 12 33 https://www.gastronom.ru/recipe/39434/ruletiki-iz-kurinogo-file-s-tykvoj
1 12 34 https://www.gastronom.ru/recipe/39431/myasnoj-rulet
1 12 35 https://www.gastronom.ru/recipe/33808/nachinka-dlya-blinov-s-kurinoj-pechenkoj
1 12 36 https://www.gastronom.ru/recipe/39403/zakuska-iz-ogurcov-i-syra
1 13 1 https://www.gastronom.ru/recipe/39393/zakuska-iz-krevetok
1 13 2 https://www.gastronom.ru/recipe/39391/zakuska-iz-rybno-krevetochnogo-sufle
1 13 3 https://www.gastronom.ru/recipe/39389/marinovannyj-syr
1 13 4 https://www.gastronom.ru/recipe/39388/glazunya-serdce-v-kusochke-hleba
1 13 5 https://www.gastronom.ru/recipe/33811/nachinka-dlya-blinchikov-s-ryboj
1 13 6 https://www.gastronom.ru/recipe/39378/Кокосовые креветки
1 13 7 https://www.gastronom.ru/recipe/39376/shashlychki-iz-krevetok-s-cherri
1 13 8 https://www.gastronom.ru/recipe/39362/Тосты с баклажаном, мятой и шпинатом
1 13 9 https://www.gastronom.ru/recipe/39361/Спринг-роллы с уткой
1 13 10 https://www.gastronom.ru/recipe/39357

...

9 13 25 https://www.gastronom.ru/recipe/29450/pyanyj-tiramisu
9 13 26 https://www.gastronom.ru/recipe/29412/milkshejk-tiramisu
https://www.gastronom.ru/recipe/28829/abrikosovyj-chizkejk-zapekanka 2
9 13 27 https://www.gastronom.ru/recipe/29369/chizkejk-s-yagodami
9 13 28 https://www.gastronom.ru/recipe/29240/tort-s-klubnikoj
9 13 29 https://www.gastronom.ru/recipe/29343/shokoladnyj-spiralnyj-tort-s-malinoj-i-ezhevikoj
9 13 30 https://www.gastronom.ru/recipe/29342/tort-belye-nochi-s-klubnichnym-kremom-sufle
9 13 31 https://www.gastronom.ru/recipe/29341/vafelnyj-tort-s-malinoj-bez-vypechki
9 13 32 https://www.gastronom.ru/recipe/29347/tort-yubilejnyj
9 13 33 https://www.gastronom.ru/recipe/29106/chizkeik-s-krasnoj-smorodinoj
9 13 34 https://www.gastronom.ru/recipe/29094/shokoladnyj-fondan-s-vishnej
9 13 35 https://www.gastronom.ru/recipe/29050/shokoladnyj-tort-s-yagodami
9 14 1 https://www.gastronom.ru/recipe/29184/tort-s-paukami
9 14 2 https://www.gastronom.ru/recipe/27693/limonnyj-chiz

9 16 28 https://www.gastronom.ru/recipe/26170/postnyj-banano-kokosovyj-tort
9 16 29 https://www.gastronom.ru/recipe/26068/domashnee-tiramisu
9 16 30 https://www.gastronom.ru/recipe/25921/blinnyj-tortik-russkaya-ekzotika
9 16 31 https://www.gastronom.ru/recipe/25912/shokoladnyj-tort-«kljukva-v-snegu»
9 16 32 https://www.gastronom.ru/recipe/25869/yablochnyj-napoleon-s-koricej
9 16 33 https://www.gastronom.ru/recipe/25834/blinnyj-tort
9 16 34 https://www.gastronom.ru/recipe/25790/blinnyj-tort
9 16 35 https://www.gastronom.ru/recipe/25757/krepvill-s-kremom-boston-i-shokoladnym-ganashem
9 16 36 https://www.gastronom.ru/recipe/25725/tort-iz-kartofelya-so-smetannym-kremom
9 17 1 https://www.gastronom.ru/recipe/25678/blinnyj-tortik
9 17 2 https://www.gastronom.ru/recipe/25677/blinnyj-tortik-izbushka
9 17 3 https://www.gastronom.ru/recipe/25671/blinnyj-tort
9 17 4 https://www.gastronom.ru/recipe/25623/serdechno-blinnyj-tort-dlya-vlyublennyh
9 17 5 https://www.gastronom.ru/recipe/25552/tort-s-ab

9 20 2 https://www.gastronom.ru/recipe/17629/yagodnyj-semifredo-s-shokoladnym-sousom
9 20 3 https://www.gastronom.ru/recipe/17626/tvorozhnyj-tort-s-mandarinami
9 20 4 https://www.gastronom.ru/recipe/17621/blinchatyj-rulet-s-shokoladom
9 20 5 https://www.gastronom.ru/recipe/17616/tort-belaya-i-chernaya-magiya
9 20 6 https://www.gastronom.ru/recipe/17613/tort-podarochnaya-korobka
9 20 7 https://www.gastronom.ru/recipe/17602/tort-pavlova
9 20 8 https://www.gastronom.ru/recipe/15773/shokoladnyj-blinnyj-tort
9 20 9 https://www.gastronom.ru/recipe/17583/orehovo-grushevyj-tort
https://www.gastronom.ru/recipe/17481/fahitas-s-kuricej-i-sladkim-percem 2
9 20 10 https://www.gastronom.ru/recipe/17356/tort-pavlova-s-klubnikoj
9 20 11 https://www.gastronom.ru/recipe/17324/apelsinovyj-chizkejk-s-kumkvatami
9 20 12 https://www.gastronom.ru/recipe/17327/tort-napoleon-s-bananami
9 20 13 https://www.gastronom.ru/recipe/11692/tiramisu
https://www.gastronom.ru/recipe/17135/napoleon-s-zelenyu 2
9 20 14 http

9 23 12 https://www.gastronom.ru/recipe/12695/tvorozhno-shokoladnyj-tort
9 23 13 https://www.gastronom.ru/recipe/12653/sicilijskij-prazdnichnyj-tort-kassata
9 23 14 https://www.gastronom.ru/recipe/12588/poleteli-italyanskij-desert-po-motivam-tiramisu
9 23 15 https://www.gastronom.ru/recipe/12486/pashalnyj-chizkejk
9 23 16 https://www.gastronom.ru/recipe/12314/napoleon-s-vetchinoj-i-ananasom
https://www.gastronom.ru/recipe/12251/blinnyj-tort-providenie 2
9 23 17 https://www.gastronom.ru/recipe/12117/cheremuhovyj-tort-s-vishnej
9 23 18 https://www.gastronom.ru/recipe/12105/klubnichnyj-tort-s-zasaharennymi-cvetami
9 23 19 https://www.gastronom.ru/recipe/12023/tort-irlandskij-krem
9 23 20 https://www.gastronom.ru/recipe/12017/tort-zhele
9 23 21 https://www.gastronom.ru/recipe/12002/tort-cherepaha
9 23 22 https://www.gastronom.ru/recipe/11911/chizkejk-v-yabloke
9 23 23 https://www.gastronom.ru/recipe/11851/tort-serdce-dlya-anny
9 23 24 https://www.gastronom.ru/recipe/11855/tiramisu-s-klubni

9 26 21 https://www.gastronom.ru/recipe/355/tort-beze-pavlova
9 26 22 https://www.gastronom.ru/recipe/2849/sicilijskaya-kassata
9 26 23 https://www.gastronom.ru/recipe/2067/shokoladnyj-chizkejk
https://www.gastronom.ru/recipe/2214/chizkejk-s-vishnyami 2
9 26 24 https://www.gastronom.ru/recipe/3986/tort-s-kokosovym-orehom-i-lajmom
9 26 25 https://www.gastronom.ru/recipe/4126/holodnyj-chizkejk-s-belym-shokoladom
9 26 26 https://www.gastronom.ru/recipe/2479/revennyj-chizkejk
9 26 27 https://www.gastronom.ru/recipe/4975/klubnichnyj-tort
9 26 28 https://www.gastronom.ru/recipe/4982/chizkejk-s-letnimi-yagodami
9 26 29 https://www.gastronom.ru/recipe/4635/italyanskij-orehovyj-tort
https://www.gastronom.ru/recipe/4705/zapekanka-iz-blinov 2
9 26 30 https://www.gastronom.ru/recipe/4880/tort-kapuchino
9 26 31 https://www.gastronom.ru/recipe/3973/tort-s-chernikoj-i-sgushchennymi-slivkami
9 26 32 https://www.gastronom.ru/recipe/4536/tort-mocart
9 27 1 https://www.gastronom.ru/recipe/4543/shokoladny

In [33]:
recipes_urls = []
resp = requests.get('https://www.gastronom.ru/recipe/group/3225/recepty-salatov')
soup = BeautifulSoup(resp.text, 'html.parser')
recs = soup.select("div.row.feed h3 > a")

for d in recs:
    recipes_urls.append('https://www.gastronom.ru'+d['href'])
recipes_urls

['https://www.gastronom.ru/recipe/31972/salat-iz-pecheni-treski-klassicheskij',
 'https://www.gastronom.ru/recipe/5136/seledka-pod-shuboj',
 'https://www.gastronom.ru/recipe/40079/klassicheskij-cezar-s-kuricej',
 'https://www.gastronom.ru/recipe/31794/salat-grecheskij-klassicheskij',
 'https://www.gastronom.ru/recipe/35696/salat-s-avokado-i-jajcom-pashot',
 'https://www.gastronom.ru/recipe/16153/salat-iz-marinovannyh-gribov-s-ovoshchami',
 'https://www.gastronom.ru/recipe/31368/salat-mimoza',
 'https://www.gastronom.ru/recipe/31321/salat-granatovyj-braslet',
 'https://www.gastronom.ru/recipe/31788/olive-klassicheskij',
 'https://www.gastronom.ru/recipe/31266/video-recipe-salat-iz-avokado-s-krevetkami-i-ogurcami',
 'https://www.gastronom.ru/recipe/9436/shvejcarskij-novogodnij-salat',
 'https://www.gastronom.ru/recipe/23246/salat-s-dikim-risom-i-avokado',
 'https://www.gastronom.ru/recipe/16074/perigorskij-salat',
 'https://www.gastronom.ru/recipe/13266/salat-uzbekiston',
 'https://www.g

In [30]:
import re
rec_text = ''
resp = requests.get('https://www.gastronom.ru/recipe/39093/kvashenaya-kapusta-bystrogo-prigotovleniya')
soup = BeautifulSoup(resp.text, 'html.parser')
title = soup.find("h1").text
steps = soup.select("div.step.instruction p")
for step in steps:
    rec_text += step.text.replace('\r\n', '').replace('\xa0', ' ')
rec_text = re.sub(r"\s+", u" ", rec_text).strip()
ingredients_collection = soup.select("li.ingredient")
ingredients = [i.text.replace('\xa0', ' ') for i in ingredients_collection]


['1 большой кочан капусты (1,8–2 кг)',
 '3 большие морковки (примерно 500 г)',
 '1 ч. л. семян тмина и/или фенхеля, по желанию',
 '120 мл яблочного или белого винного уксуса',
 '1 ст. л. с горкой сахара',
 '2 ст. л. с горкой нейодированной соли']

In [40]:
import re

with open('recipes_gastronom.csv', 'w') as f:
    f.write('name|ingredients|text\n')

n = 0

for n, url in enumerate(recipes_urls, 1):
    if n == 3:
        break
    rectext = ''
    ingredients = []
    print(n, url)
    resp_recipe = requests.get(url)
    recipe = BeautifulSoup(resp_recipe.text, 'html.parser')
    recsteps = recipe.select("div.step.instruction p")
    if len(recsteps) == 0:
        print ('0')
        continue
    name = recipe.find("h1").text
    ingredients_collection = recipe.select("li.ingredient")
    ingredients = [i.text.replace('\xa0', ' ') for i in ingredients_collection]
    for step in recsteps:
        rectext += step.text.replace('\r\n', '').replace('\xa0', ' ')
    rectext = re.sub(r"\s+", u" ", rectext).strip()
    with open('recipes_gastronom.csv', 'a') as f:
        f.write('{}|{}|{}\n'.format(name, ','.join(ingredients), rectext))
    n += 1
    time.sleep(2) 
                

1 https://www.gastronom.ru/recipe/31972/salat-iz-pecheni-treski-klassicheskij
2 https://www.gastronom.ru/recipe/5136/seledka-pod-shuboj


In [86]:
recp_recipe = requests.get('https://www.utkonos.ru/recipe/1072')
recipe = BeautifulSoup(recp_recipe.text, 'html.parser')

In [87]:
name = recipe.find("h1")
name.text

'Смузи аппетайзер '

In [88]:
rectext = ''
recsteps = recipe("div", class_= "cooking_step_desc")
for step in recsteps:
        rectext += step.text
rectext

'С огурцов снимите кожицу, удалите семена, мякоть крупно нарежьте. Яблоко очистите от семян и кожицы, нарежьте, сбрызните соком лимона, чтобы не потемнело. С сельдерея срежьте загрубевшие места   и жесткие волокна, нарежьте некрупными ломтиками. В блендер поместите кусочки огурца, сельдерея, яблока, добавьте укроп, соль, оставшийся сок лимона. Все взбейте и подавайте в охлажденных бокалах. '

In [89]:
ingredients_collection = reciepe.select("dd")
del ingredients_collection[len(ingredients_collection) - 1]
ingredients = [i.text.split('—')[0].strip() for i in ingredients_collection]
ingredients

['Огурцы', 'Яблоки зеленые', 'Сельдерей', 'Сок', 'Укроп', 'Соль']